# 미니배치 경사 하강법 (Mini-batch Gradient Descent)

**미니배치 경사 하강법(Mini-batch Gradient Descent)**은 전체 데이터셋을 작은 그룹, 즉 **미니배치(mini-batch)**로 나누어 각 미니배치에 대한 그래디언트를 계산하고 파라미터를 업데이트하는 최적화 알고리즘임.

이 방법은 전체 데이터를 사용하는 배치 경사 하강법(Batch Gradient Descent)과 한 개의 샘플만 사용하는 확률적 경사 하강법(SGD)의 절충안임. 배치 경사 하강법의 안정적인 그래디언트 추정과 SGD의 빠른 계산 속도라는 두 가지 장점을 모두 취할 수 있어, 현대 딥러닝에서 가장 표준적으로 사용되는 방식임.

### 📐 수학적 배경

미니배치 경사 하강법의 업데이트 규칙은 비용 함수 $J$와 그래디언트 $\nabla J$를 크기가 $m$인 미니배치 $B$에 대해 계산함.

$$ \theta_{\text{new}} := \theta_{\text{old}} - \eta \nabla J(\theta; X^{(B)}, y^{(B)}) $$

* $\theta$: 업데이트할 모델 파라미터 (가중치 $W$, 편향 $b$)
* $\eta$: 학습률(Learning Rate)
* $X^{(B)}, y^{(B)}$: $m$개의 샘플로 구성된 현재 미니배치 데이터
* $\nabla J(...)$: 미니배치에 대한 평균 그래디언트

#### 예시: 선형 회귀의 미니배치 그래디언트
미니배치에 대한 비용 함수(MSE) $J = \frac{1}{m} \sum_{j=1}^{m}((Wx^{(i_j)} + b) - y^{(i_j)})^2$ 에 대해, 각 파라미터의 그래디언트는 다음과 같이 미니배치에 대한 평균으로 계산됨.

$$ \frac{\partial J}{\partial W} = \frac{2}{m} \sum_{j=1}^{m} ((Wx^{(i_j)} + b) - y^{(i_j)})x^{(i_j)} $$
$$ \frac{\partial J}{\partial b} = \frac{2}{m} \sum_{j=1}^{m} ((Wx^{(i_j)} + b) - y^{(i_j)}) $$

### 셋업 (Setup)
계산 및 시각화를 위한 라이브러리 임포트.

In [ ]:
# !pip install numpy matplotlib
import numpy as np
import matplotlib.pyplot as plt

### 예제 데이터 준비 (Data Preparation)
SGD 예제와 동일한 데이터를 사용하여 결과를 비교하기 용이하게 함. 목표는 $y \approx 3X + 4$ 관계를 찾아내는 것.

In [ ]:
# 100개의 데이터 포인트 생성
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1) # y = 4 + 3X + noise

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")

### 핵심 로직 구현 (Code Implementation)
미니배치 경사 하강법 알고리즘을 구현하여 선형 회귀 모델의 가중치(W)와 편향(b)을 학습시킴.

In [ ]:
# 1. 파라미터 초기화
W = np.random.randn()
b = np.random.randn()

# 2. 하이퍼파라미터 설정
learning_rate = 0.01
epochs = 50
batch_size = 16 # 미니배치 크기 설정
n = len(X)

cost_history = []

# 3. 미니배치 학습 시작
for epoch in range(epochs):
    # 매 에포크마다 데이터 순서를 섞음
    shuffled_indices = np.random.permutation(n)
    X_shuffled = X[shuffled_indices]
    y_shuffled = y[shuffled_indices]
    
    # 3-1. 미니배치 단위로 학습 진행
    for i in range(0, n, batch_size):
        # 미니배치 데이터 슬라이싱
        xi = X_shuffled[i:i+batch_size]
        yi = y_shuffled[i:i+batch_size]
        m = len(xi)
        
        # 예측값 계산
        y_pred = W * xi + b
        
        # 미니배치에 대한 평균 그래디언트 계산
        grad_W = (2/m) * np.sum((y_pred - yi) * xi)
        grad_b = (2/m) * np.sum(y_pred - yi)
        
        # 파라미터 업데이트
        W = W - learning_rate * grad_W
        b = b - learning_rate * grad_b
        
    # 3-2. 매 에포크 종료 시 전체 데이터에 대한 비용 계산 및 기록
    total_cost = np.mean((W * X + b - y)**2)
    cost_history.append(total_cost)
    
    # 학습 과정 출력 (5번마다)
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1:2d} | Cost: {total_cost:.4f} | W: {W:.4f} | b: {b:.4f}")

### 📊 결과 확인 및 시각화 (Results & Visualization)

#### 비용 함수 변화 시각화
미니배치의 비용 감소는 SGD보다 안정적이지만, 배치 경사 하강법보다는 변동성이 있는 모습을 보임.

In [ ]:
plt.plot(cost_history)
plt.title("Cost History over Epochs (Mini-batch GD)")
plt.xlabel("Epochs")
plt.ylabel("Cost (MSE)")
plt.show()

#### 최종 회귀선 시각화

In [ ]:
print(f"\n최종 학습 결과 -> W: {W:.4f}, b: {b:.4f} (정답 근사치: W=3, b=4)")

plt.scatter(X, y, label='Original Data', alpha=0.7)
plt.plot(X, W * X + b, color='red', label='Fitted Line (Mini-batch GD)')
plt.title("Data and Fitted Regression Line")
plt.xlabel("X")
plt.ylabel("y")
plt.legend()
plt.show()

### 고려사항 (Considerations)

* **배치 크기(Batch Size)**: 미니배치 경사 하강법에서 가장 중요한 하이퍼파라미터 중 하나임.
  * **작은 배치 크기**: 학습이 불안정하지만(noisy) 일반화 성능이 좋아지는 경향이 있음. SGD의 특성과 유사함.
  * **큰 배치 크기**: 학습이 안정적이지만 지역 최솟값에 빠질 위험이 있고, 더 많은 메모리를 요구함. 배치 경사 하강법의 특성과 유사함.
  * 일반적으로 2의 거듭제곱(예: 16, 32, 64, 128)을 배치 크기로 사용하는 것이 GPU 하드웨어의 메모리 구조에 효율적이라고 알려져 있음.

* **경사 하강법 비교**
| 구분 | 배치(Batch) GD | 확률적(Stochastic) GD | 미니배치(Mini-batch) GD |
| :--- | :--- | :--- | :--- |
| **업데이트 당 데이터** | 전체 데이터 | 1개 | $m$개 (미니배치 크기) |
| **업데이트 속도** | 느림 | 빠름 | 중간 |
| **그래디언트 노이즈** | 없음 (안정적) | 큼 (불안정) | 중간 |
| **메모리 사용량** | 큼 | 작음 | 중간 |
| **주요 특징** | 안정적인 수렴 | 지역 최솟값 탈출 가능 | 속도와 안정성의 균형 |


### 최종 요약 (Conclusion)

미니배치 경사 하강법은 안정성과 효율성 사이의 균형을 맞춘 알고리즘으로, 현대 딥러닝 모델 학습의 표준적인 방법으로 자리 잡았음. 전체 데이터를 작은 묶음(미니배치)으로 나누어 학습함으로써, 대용량 데이터셋에서도 빠르고 안정적으로 최적의 파라미터를 찾을 수 있음.

이 노트북에서는 미니배치 경사 하강법을 직접 구현하고, 비용 함수가 SGD보다는 안정적이고 배치 경사 하강법보다는 빠르게 수렴하는 과정을 확인함. 배치 크기가 학습 과정과 성능에 미치는 영향을 이해하는 것이 중요함.